%md
# FNB Silver – Cleaned & Standardized Layer

This notebook reads the Bronze FNB raw data, performs cleaning, standardization,  
type casting, and deduplication, and publishes the curated Silver-layer dataset.

Data Source:

| Tables |
| ------ |
| default.bronze_fnb_sales |

Target Tables:

| Tables |
| ------ |
| default.silver_fnb_sales |


In [0]:
dbutils.notebook.exit(1)

### Imports

In [0]:
from pyspark.sql.functions import cast, col, current_timestamp

### Read from bronze

In [0]:
df_bronze = spark.read.table('default.bronze_fnb_sales')

### Transformations

#### Type Casting

In [0]:
df_bronze_altered = (
    df_bronze.withColumn("SALES", col("SALES").try_cast("double"))
    .withColumn("UNITS", col("UNITS").try_cast("int"))
    .withColumn("VOLUME", col("VOLUME").try_cast("double"))
    # .withColumn("AVERAGE_UNIT_PRICE", col("AVERAGE_UNIT_PRICE").try_cast("double"))
    .withColumn("DISTRIBUTION", col("DISTRIBUTION").try_cast("double"))
    .withColumn("PROMO_SALES", col("PROMO_SALES").try_cast("double"))
    .withColumn("PROMO_UNITS", col("PROMO_UNITS").try_cast("double"))
    .withColumn("DATE", col("DATE").try_cast("date"))
    .withColumn("LAST_UPDATED", current_timestamp())
)

#### Currency Conversion

In [0]:
df_silver = df_bronze_altered.withColumn(
    "SALES_INR", (col("SALES") * 88.44).try_cast("double")
).withColumn("SALES_GBP", (col("SALES") * 0.7605).try_cast("double"))

### Save as Table

In [0]:
df_silver_final = df_silver.select(
    "COUNTRY",
    "RETAILER_ID",
    "RETAILER",
    "PRODUCT_ID",
    "ITEM_CODE",
    "CATEGORY",
    "SEGMENT",
    "BRAND",
    "MANUFACTURER",
    "SALES",
    "UNITS",
    "VOLUME",
    "DISTRIBUTION",
    "PROMO_SALES",
    "PROMO_UNITS",
    # "AVERAGE_UNIT_PRICE",
    "CURRENCY_CODE",
    "SALES_INR",
    "SALES_GBP",
    "DATE",
    "LAST_UPDATED",
)

In [0]:
df_silver_final.count()

In [0]:
# df_silver_final.write.format('delta').mode('overwrite').option("mergeSchema", "true").partitionBy('CATEGORY','RETAILER').saveAsTable('default.silver_fnb_sales')